# Nanosafety data annotation

In [3]:
# Setup
import ipywidgets as widgets
import json
import os,datetime

import logging
from logging.config import fileConfig
fileConfig('logging_endpoints_config.ini')
global logger
logger = logging.getLogger()
logger.debug('Started at %s \t%s',os.name, datetime.datetime.now())

try:
    with open('./config.solr.json') as f:
        config = json.load(f)
    solr_services=config["services"]    
    solr_query=config["query"]   
except Exception as err:        
    config = None
    logger.error("Something went wrong",err)
service_widget = widgets.Dropdown(
    options=solr_services.keys(),
    description='Service:',
    disabled=False,
)
display(service_widget)

Dropdown(description='Service:', options=('NANoREG', 'eNanoMapper', 'NanoReg2', 'caLIBRAte', 'GRACIOUS', 'PATR…

In [4]:
apikey_widget=widgets.Text(
    placeholder='',
    description='API key',
    disabled=False
)
display(apikey_widget)


Text(value='', description='API key', placeholder='')

In [5]:
term_widget = widgets.Dropdown(
    options=solr_query.keys(),
    description='term:',
    disabled=False,
)

display(term_widget)

Dropdown(description='term:', options=('substancetype', 'nanomaterial', 'protocol', 'endpointcategory', 'endpo…

In [6]:
project=service_widget.value
solr_url=solr_services[project]
#nested facet
synonym_facet=",facet: {field2: {"+ "{}:{},{}:{} ,limit : -1, mincount:1 ".format("type","terms","field",solr_query[term_widget.value]["synonym"]) +"}}"
json_facet="{field1: {" + "{}:{},{}:{} ,limit : -1, mincount:1 ".format("type","terms","field",solr_query[term_widget.value]["field"]) + synonym_facet + " }}"

query={'q': solr_query[term_widget.value]["q"],"wt" : "json", "json.facet": json_facet, 'rows': 0}
logger.debug(solr_url)
logger.info(query)

2019-03-24 13:10:59,497  INFO     {'q': 'type_s:params', 'wt': 'json', 'json.facet': '{field1: {type:terms,field:E.cell_type_s ,limit : -1, mincount:1 ,facet: {field2: {type:terms,field:E.cell_type_s ,limit : -1, mincount:1 }} }}', 'rows': 0}


In [7]:
import requests
import enmapi   
  
r = requests.post(solr_url,data=query, auth=enmapi.AmbitAPIKEYAuth(apikey_widget.value) ,stream=False)
logger.info(r.status_code)
#print(r.json())

2019-03-24 13:11:00,055  INFO     200


In [9]:
import pandas as pd
key1 = "field1"
key2 = "field2"

fields = r.json()["facets"][key1]["buckets"]
fields_name=[]
fields_count=[]
field_synonym=[]
for value in fields:
    fields_name.append(value['val'])
    fields_count.append(value['count'])
    _synonym=''
    try:
        for synonym in value[key2]['buckets']:
            if synonym['val'].startswith("EP_"):
                continue
            _synonym = _synonym + " " + synonym['val']

    except Exception as err:
        pass
    field_synonym.append(_synonym.strip())    

df = pd.DataFrame({term_widget.value : fields_name, "count" : fields_count, key2 : field_synonym})


In [10]:
propertiesfile_widget=widgets.Text(
    placeholder='',
    description='Annotation file',
    disabled=False
    
)
display(propertiesfile_widget)


Text(value='', description='Annotation file', placeholder='')

In [18]:
import os, os.path
from jproperties import Properties

prop_file=solr_query[term_widget.value]['annotation']
if propertiesfile_widget.value == '':
    prop_file='./annotation/'+ prop_file
else:
    prop_file= os.path.normcase(os.path.join(propertiesfile_widget.value,prop_file))

p = Properties()
try:
    with open(prop_file, "rb") as f:
        p.load(f, "utf-8") 

except Exception as err:
    p = None
    print(err)
print(prop_file)

./annotation/cells.properties


In [19]:
def _annotate(x):
    if p is None:
        return None
    x=x.replace(" ","_").upper().strip()
    
    if x in p:
        value, meta = p[x]
        return value
    else:
        return None
    

df["_annotation"] = df[term_widget.value].apply(lambda x: ( _annotate(x)))
df[[term_widget.value,"_annotation"]].head()

In [ ]:
def store(p):
    if p != None:
        with open('./annotation/' + solr_query[term_widget.value]['annotation'], "wb") as f:
            p.store(f, encoding="utf-8")
#store(p)            